In [1]:
from ddpipe.client import DDClient
import time

In [2]:
dd = DDClient()
now = int(time.time())
hour_ago = now - 3600

In [3]:
df = dd.query("avg:system.cpu.user{*} by {host}", since=hour_ago, until=now)
print(df.head())

            timestamp     value           metric       host           scope
0 2025-10-14 12:42:40  6.158948  system.cpu.user  DZ_Laptop  host:DZ_Laptop
1 2025-10-14 12:43:00  9.244552  system.cpu.user  DZ_Laptop  host:DZ_Laptop
2 2025-10-14 12:43:20  9.413847  system.cpu.user  DZ_Laptop  host:DZ_Laptop
3 2025-10-14 12:43:40  6.380530  system.cpu.user  DZ_Laptop  host:DZ_Laptop
4 2025-10-14 12:44:00  8.059972  system.cpu.user  DZ_Laptop  host:DZ_Laptop
